### Load FaceNet Model

[Model](https://github.com/nyoki-mtl/keras-facenet)

In [12]:
import numpy as np
import tensorflow as tf
import os

In [13]:
from tensorflow.python.lib.io import file_io

model_file = file_io.FileIO('gs://shield-phishing/face_verification/pretrained_models/face_net_nyoki/facenet.h5', mode='rb')
with open("./temp_face_model.h5",'wb') as f:
    f.write(model_file.read())
    model_file.close()
    model = tf.keras.models.load_model(f.name)
    os.remove(f.name)

print(model.inputs)
print(model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [ ]:
from tensorflow import keras

model = keras.models.load_model('./inception_model/hiroki/facenet_keras.h5')

print(model.inputs)
print(model.outputs)

In [2]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)

    return embedding / np.linalg.norm(embedding, ord=2)  # rescale the encoding vectors to have L2 norm equal to one

In [3]:
def verify(encoding_1, encoding_2):
    # Compute distance
    dist = np.linalg.norm(encoding_1 - encoding_2)

    if dist < 0.7:
        verified = True
    else:
        verified = False
    print("dist", dist)

    return verified

In [10]:
print(verify(img_to_encoding("./tutorial/images/camera_1.jpg", model), img_to_encoding("./tutorial/images/bertrand.jpg", model)))
print(verify(img_to_encoding("./tutorial/images/camera_3.jpg", model), img_to_encoding("./tutorial/images/bertrand.jpg", model)))
print(verify(img_to_encoding("./tutorial/images/camera_1.jpg", model), img_to_encoding("./tutorial/images/younes.jpg", model)))

(0.5436483, True)
(0.386163, True)
(1.3963861, False)


#### Ways to improve your facial recognition model

- Put more images of each person (under different lighting conditions, taken on different days, etc.) into the database. Then, given a new image, compare the new face to multiple pictures of the person. This would increase accuracy.
- Crop the images to contain just the face, and less of the "border" region around the face. This preprocessing removes some of the irrelevant pixels around the face, and also makes the algorithm more robust.